# Codeado por Franco M. Di Maria

# Guia 2 : Spark

## Funciones auxiliares 

In [1]:
import datetime
FORMATO_FECHA = '%Y-%m-%d'

In [2]:
import random
import time

# By Tom Alsberg and Boris (StackOverflow) 
def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end):
    return str_time_prop(start, end, FORMATO_FECHA, random.random())

## Ejercicio 1

1-Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota, fecha). Se pide resolver utilizando PySpark:
* A.Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.
* B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).
* C.El nombre y apellido del alumno con mejor promedio. Para esto pued eutilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).

### Set de datos

In [3]:
pre_data_1_notas = [
    (100498, 'Datos', 10, '2019-9-16'),
    (100499, 'Datos', 9, '2019-9-17'),
    (100500, 'Datos', 8, '2019-9-18'),
    (100501, 'Datos', 7, '2019-9-19'),
    (100502, 'Datos', 7, '2019-9-20'),
    (100498, 'Taller', 9, '2019-6-1'),
    (100499, 'Taller', 6, '2019-6-2'),
    (100500, 'Taller', 8, '2019-6-3'),
    (100501, 'Taller', 8, '2019-6-4'),
    (100502, 'Taller', 9, '2019-6-5'),
    (100498, 'Proba', 8, '2018-11-11'),
    (100499, 'Proba', 6, '2018-11-12'),
    (100500, 'Proba', 6, '2018-11-13'),
    (100501, 'Proba', 7, '2018-11-14'),   
    (100502, 'Proba', 8, '2018-11-15'),
]

rdd_1 = sc.parallelize(pre_data_1_notas)
#rdd_1.collect()

### A.Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.

##### RESOLUCION 1

In [4]:
rdd_1.filter( lambda x: (x[2] >= 4) & (x[3] > "2018-1-1") & (x[3] < "2018-12-31") )\
    .map( lambda x: x[0] )\
    .distinct()\
    .count()

5

##### RESOLUCION 2

In [5]:
rdd_1.filter( lambda x: (x[2] >= 4) & (x[3] > "2018-1-1") & (x[3] < "2018-12-31") )\
    .flatMap( lambda x: (x[0],) )\
    .distinct()\
    .count()

5

##### RESOLUCION 3

In [6]:
rdd_1.filter( lambda x: (x[2] >= 4) & (x[3] > "2018-1-1") & (x[3] < "2018-12-31") )\
    .map( lambda x: (x[0],1) )\
    .reduceByKey( lambda x, y: x )\
    .count()

5

### B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).

In [7]:
rdd_1_promedios = rdd_1.map( lambda x: (x[0], (x[2], 1)) )\
                    .reduceByKey( lambda x, y: (x[0] + y[0], x[1] + y[1]) )\
                    .map( lambda x: (x[0], x[1][0] / x[1][1]) )
rdd_1_promedios.collect()

[(100498, 9.0),
 (100499, 7.0),
 (100500, 7.333333333333333),
 (100501, 7.333333333333333),
 (100502, 8.0)]

### C.El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).

#### Set de datos auxiliar

In [8]:
pre_data_1_alumnos = [
    (100498, 'Franco Martin', 'Di Maria'),
    (100499, 'Alumno B', 'Ble Ble'),
    (100500, 'Alumno C', 'Bli Bli'),
    (100501, 'Alumno D', 'Blo Blo'),
    (100502, 'Alumno E', 'Blu Blu'),
]

rdd_1_alumnos = sc. parallelize(pre_data_1_alumnos)

##### RESOLUCION 1 

In [9]:
rdd_1_alumnos_por_padron = rdd_1_alumnos.map( lambda x: (x[0], x[1] + ' ' + x[2]) )
padron_promedio_max = rdd_1_promedios.reduce(lambda x, y: x if x[1] > y[1] else y)
rdd_1_alumnos_por_padron.filter(lambda x: x[0] == padron_promedio_max[0])\
    .map(lambda x: (x[0], x[1], padron_promedio_max[1]) )\
    .take(1)

[(100498, 'Franco Martin Di Maria', 9.0)]

##### RESOLUCION 2

In [10]:
rdd_1_alumnos_por_padron = rdd_1_alumnos.map( lambda x: (x[0], x[1] + ' ' + x[2]) )
rdd_1_alumnos_por_padron.join(rdd_1_promedios)\
    .map( lambda x: (x[0], x[1][0], x[1][1]) )\
    .reduce( lambda x, y: x if x[2] > y[2] else y )\

(100498, 'Franco Martin Di Maria', 9.0)

## Ejercicio 2

2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de
producto, precio de venta) y en otro RDD detalle de los productos con (código de producto,
descripción del producto, categoría). Se pide resolver utilizando PySpark:
* A. Cuál es el producto más vendido.
* B. Cuál es la categoría de productos más vendida.
* C. Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)
* D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.
* E. Idem anterior, pero calculando la categoría de productos que registró mayor variación de precios en el último año.

### Sets de datos 

In [11]:
pre_data_2_ventas = []
pre_data_2_detalles = []
cant_productos = 10
codigo_productos = [i for i in range(cant_productos)]
for _ in range(500):
    pre_data_2_ventas.append( 
        ( random_date('2018-1-1', '2019-12-31'), random.choice(codigo_productos), random.uniform(100, 1000) ) 
    )
for i in range(cant_productos):
    pre_data_2_detalles.append( 
        ( i, 'Descripcion ...' , random.choice(['Categ. A', 'Categ. B', 'Categ. C']) ) 
    )
rdd_2_ventas = sc.parallelize(pre_data_2_ventas)
rdd_2_detalles = sc.parallelize(pre_data_2_detalles)

### A. Cuál es el producto más vendido. 

In [12]:
rdd_2_ventas.map( lambda x: (x[1], 1) )\
    .reduceByKey( lambda x, y: x + y )\
    .reduce(lambda x, y: x if x[1] > y[1] else y)[0]

2

In [13]:
rdd_2_producto_cant = rdd_2_ventas.map( lambda x: (x[1], 1) )\
                        .reduceByKey(lambda x, y: x + y)
rdd_2_producto_cant.takeOrdered(1, lambda x: -x[1])[0][0] # Pues devuelve una lista

2

In [14]:
rdd_2_producto_cant.collect()

[(0, 56),
 (8, 51),
 (1, 44),
 (9, 49),
 (2, 57),
 (3, 53),
 (4, 56),
 (5, 38),
 (6, 45),
 (7, 51)]

Puede pasar que haya dos productos con igual cantidad de ventas, por lo que las dos operaciones anteriores puede dar distinto.

### B. Cuál es la categoría de productos más vendida.

In [15]:
# Nos quedamos con rdd (codigo_producto, cantidad)
rdd_2_producto_cant = rdd_2_ventas.map( lambda x: (x[1], 1) )\
                        .reduceByKey(lambda x, y: x + y)

# Nos quedamos con un rdd (codigo_producto, categoria)
rdd_2_producto_categ = rdd_2_detalles.map( lambda x: (x[0], x[2]) )

# Joineamos a partir de la misma clave : codigo_producto, obteniedo un rdd : 
# ( codigo_producto, (cantidad, categoria) )
rdd_2_producto_cant_categ = rdd_2_producto_cant.join(rdd_2_producto_categ)

# Llevamos la categoria a la clave y nos quedamos solo con la cantidad de 
# repeticiones de esa categoria para cada producto. Como se pueden repetir
# la misma categoria para mas de un producto, debemos sumar las cantidades
# para una misma categoria, y luego reducir para quedarno con la categoria 
# mas vendida
rdd_2_producto_cant_categ.map( lambda x: (x[1][1], x[1][0]) )\
    .reduceByKey(lambda x, y: x + y)\
    .reduce(lambda x, y: x if x[1] > y[1] else y)[0]

'Categ. C'

### C. Cuál es el top 5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)

In [16]:
rdd_2_producto_descripcion = rdd_2_detalles.map(lambda x: (x[0], x[1]) )

rdd_2_producto_cant_top_5 = sc.parallelize(rdd_2_producto_cant.takeOrdered(5, lambda x: -x[1]))

rdd_2_producto_cant_descripcion = rdd_2_producto_cant_top_5.join(rdd_2_producto_descripcion)

rdd_2_producto_cant_descripcion.map(lambda x: (x[0], x[1][1], x[1][0]) )\
    .takeOrdered(5, lambda x: -x[2])

[(2, 'Descripcion ...', 57),
 (0, 'Descripcion ...', 56),
 (4, 'Descripcion ...', 56),
 (3, 'Descripcion ...', 53),
 (8, 'Descripcion ...', 51)]

### D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año. 

Tomemos 5 en lugar de 50, por el set que tenemos

In [17]:
minima_cant_ventas = 5 # 50
rdd_2_productos_considerar = rdd_2_ventas.filter(lambda x:  (x[0] < '2019-10') & (x[0] > '2018-10') )\
                                .map(lambda x: (x[1], 1 ) )\
                                .reduceByKey(lambda x, y: (x + y))\
                                .filter(lambda x: x[1] >= minima_cant_ventas)

rdd_2_productos_considerar_completo = rdd_2_ventas.map(lambda x: (x[1], (x[0], x[2]) ) )\
                                        .join(rdd_2_productos_considerar)\
                                        .map(lambda x: (x[0], (x[1][0][0], x[1][0][1]) ) )
# (codigo_producto, ( (fecha, precio), cantidad ) ) => (codigo_producto, (fecha, precio) )


rdd_2_precios_finales = rdd_2_productos_considerar_completo\
                            .reduceByKey(lambda vx, vy: vx if vx[0] > vy[0] else vy)\
                            .map(lambda x: (x[0], x[1][1])) # (codigo_producto, precio) # finales

rdd_2_precios_iniciales = rdd_2_productos_considerar_completo\
                            .reduceByKey(lambda vx, vy: vx if vx[0] < vy[0] else vy)\
                            .map(lambda x: (x[0], x[1][1]) ) # (codigo_producto, precio) # iniciales


rdd_2_precio_diff_porcental = rdd_2_precios_iniciales.join(rdd_2_precios_finales)
# (codigo_producto, (precio_inicial, precio_final) )


rdd_2_precio_diff_porcental.map(lambda x: (x[0], x[1][1] * 100 / x[1][0]))\
    .reduce(lambda x, y: x if x[1] > y[1] else y)

(0, 515.3526455511341)

### E. Idem anterior, pero calculando la categoría de productos que registró mayor variación de precios en el último año.

In [18]:
rdd_2_categorias_considerar = rdd_2_productos_considerar_completo.join(rdd_2_detalles.map(lambda x: (x[0], x[2])))\
                                .map(lambda x: (x[1][1], (x[1][0]) ))
# (codigo_producto, ( (fecha, precio), categoria ) ) => (categoria, (fecha, precio))

rdd_2_precios_categ_final = rdd_2_categorias_considerar.reduceByKey(lambda vx, vy: vx if vx[0] > vy[0] else vy)\
                                .map(lambda x: (x[0], x[1][1]) )

rdd_2_precios_categ_inicial = rdd_2_categorias_considerar.reduceByKey(lambda vx, vy: vx if vx[0] < vy[0] else vy)\
                                .map(lambda x: (x[0], x[1][1]) )
# (categoria, precio) # en ambos de arriba

rdd_2_precio_categ_diff_porcental = rdd_2_precios_categ_inicial.join(rdd_2_precios_categ_final)
# (categoria, (precio_inicial, precio_final))

rdd_2_precio_categ_diff_porcental.map(lambda x: (x[0], x[1][1] * 100 / x[1][0]) )\
    .reduce(lambda x, y: x if x[1] > y[1] else y)


('Categ. C', 559.3095975782546)

## Ejercicio 3

3- Se tiene un RDD con información de vuelos programados con la forma (número de vuelo, código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM). A su vez, se cuenta con el registro actualizado del estado de los vuelos que fueron ocurriendo, con la forma (número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado). En base al estado, podría contar con algún dato en blanco, por ejemplo si el vuelo fue cancelado no tendrá información de fechas y horas, si el vuelo se encuentra aún en curso, no contendrá información de la llegada. Se pide resolver utilizando PySpark:
* A. Cuál es el aeropuerto con mayor tránsito.
* B. Cuál es la aerolínea con mayor cantidad de vuelos.
* C. Cuál es la aerolínea con mayor cantidad de cancelaciones.
* D. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de salida.
* E. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de llegada.
* F. Cuál es la aerolínea más puntual.
* G. Cuál es el aeropuerto que registra mayor desviación con respecto a los horarios coordinados.

### Set de datos 

In [19]:
dia = datetime.datetime.strptime('2019-1-1', FORMATO_FECHA)
dia += datetime.timedelta(days=1)
dia.strftime(FORMATO_FECHA)

'2019-01-02'

In [20]:
CANCELADO = 'CANCELADO'
EN_CURSO = 'EN CURSO'
FINALIZADO = 'FINALIZADO'
pre_data_3_programa_vuelos = [
    ('Vuelo 0', 'Aerolinea 0', 'Aeropuerto Salida 0', 'Aeropuerto llegada 4', '20190113', '02:00','20190113', '04:00'),
    ('Vuelo 1', 'Aerolinea 0', 'Aeropuerto Salida 1', 'Aeropuerto llegada 4', '20190214', '02:30','20190114', '04:00'),
    ('Vuelo 2', 'Aerolinea 0', 'Aeropuerto Salida 1', 'Aeropuerto llegada 5', '20190315', '06:00','20190115', '12:00'),
    ('Vuelo 3', 'Aerolinea 0', 'Aeropuerto Salida 1', 'Aeropuerto llegada 5', '20190416', '17:00','20190116', '18:30'),
    ('Vuelo 4', 'Aerolinea 1', 'Aeropuerto Salida 2', 'Aeropuerto llegada 6', '20190517', '20:30','20190117', '22:00'),
    ('Vuelo 5', 'Aerolinea 1', 'Aeropuerto Salida 3', 'Aeropuerto llegada 7', '20190618', '23:30','20190118', '01:00'),
]

pre_data_3_estado_vuelos = [
    ('Vuelo 0', 'Aerolinea 0', '20190113', '02:00','20190113', '05:00', FINALIZADO),
    ('Vuelo 1', 'Aerolinea 0', '20190214', '03:00','20190114', '04:30', FINALIZADO),
    ('Vuelo 2', 'Aerolinea 0', '', '', '', '', CANCELADO),
    ('Vuelo 3', 'Aerolinea 0', '', '', '', '', CANCELADO),
    ('Vuelo 4', 'Aerolinea 1', '', '', '', '', CANCELADO),
    ('Vuelo 5', 'Aerolinea 1', '20190619', '00:30','', '', EN_CURSO),
]

rdd_3_programa = sc.parallelize(pre_data_3_programa_vuelos)
rdd_3_estado = sc.parallelize(pre_data_3_estado_vuelos)

### A. Cuál es el aeropuerto con mayor tránsito.

In [21]:
# Necesito aeropuerto salida y llegada
rdd_3_aeropuerto_salida = rdd_3_programa.map( lambda x: (x[2], 1) )\
                            .reduceByKey(lambda x, y: x + y)

rdd_3_aeropuerto_llegada = rdd_3_programa.map( lambda x: (x[3], 1) )\
                            .reduceByKey(lambda x, y: x + y)

rdd_3_aeropuerto = rdd_3_aeropuerto_salida.union(rdd_3_aeropuerto_llegada)

rdd_3_aeropuerto.reduceByKey(lambda x, y: x + y)\
    .reduce(lambda x, y: x if x[1] > y[1] else y)[0]


'Aeropuerto Salida 1'

### B. Cuál es la aerolínea con mayor cantidad de vuelos.

In [22]:
rdd_3_programa.map( lambda x: (x[1], 1) )\
    .reduceByKey(lambda x, y: x + y)\
    .reduce(lambda x, y: x if x[1] > y[1] else y)[0]

'Aerolinea 0'

### C. Cuál es la aerolínea con mayor cantidad de cancelaciones. 

In [23]:
rdd_3_estado.filter(lambda x: x[6] == CANCELADO)\
    .map(lambda x: (x[1], 1))\
    .reduceByKey(lambda x, y: x + y)\
    .reduce(lambda x, y: x if x[1] > y[1] else y)[0]

'Aerolinea 0'

### D. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de salida.

In [24]:
rdd_3_horario_salida_teor = rdd_3_programa.map(lambda x: (
                                                x[0], 
                                                datetime.datetime.combine(
                                                    datetime.datetime.strptime(x[4], '%Y%m%d').date(),
                                                    datetime.datetime.strptime(x[5], '%H:%M').time()
                                                ) 
                                            ))

rdd_3_horario_salida_real = rdd_3_estado.filter(lambda x: x[6] != CANCELADO)\
                                .map(lambda x: (
                                    x[0],
                                    datetime.datetime.combine(
                                        datetime.datetime.strptime(x[2], '%Y%m%d').date(),
                                        datetime.datetime.strptime(x[3], '%H:%M').time()
                                    )
                                ))
# Supongo que no hay vuelos repetidos con distinto valor e ninguno de los dos rdds
rdd_3_horario_salida_delta = rdd_3_horario_salida_teor.join(rdd_3_horario_salida_real)
# ('Vuelo i', (datetime_teor, datetime_real) )

rdd_3_horario_salida_delta = rdd_3_horario_salida_delta.map(lambda x: (
                                                            x[0], 
                                                            (
                                                                x[1][0].strftime('%Y-%m-%d %H:%M'), 
                                                                x[1][1] - x[1][0]
                                                            ) 
                                                        ))

# ('Vuelo i, (Fecha teor, timedelta)')
vuelo_mas_restrasado_salida = rdd_3_horario_salida_delta.reduce(lambda x, y: x if x[1][1] > y[1][1] else y)
vuelo_mas_restrasado_salida = (vuelo_mas_restrasado_salida[0], vuelo_mas_restrasado_salida[1][0])
vuelo_mas_restrasado_salida # (Vuelo i, Fecha de llegada teor)

('Vuelo 5', '2019-06-18 23:30')

### E. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de llegada. 

In [25]:
rdd_3_horario_llegada_teor = rdd_3_programa.map(lambda x: (
                                                x[0], 
                                                datetime.datetime.combine(
                                                    datetime.datetime.strptime(x[6], '%Y%m%d').date(),
                                                    datetime.datetime.strptime(x[7], '%H:%M').time()
                                                ) 
                                            ))

rdd_3_horario_llegada_real = rdd_3_estado.filter(lambda x: x[6] == FINALIZADO)\
                                .map(lambda x: (
                                    x[0],
                                    datetime.datetime.combine(
                                        datetime.datetime.strptime(x[4], '%Y%m%d').date(),
                                        datetime.datetime.strptime(x[5], '%H:%M').time()
                                    )
                                ))
# Supongo que no hay vuelos repetidos con distinto valor e ninguno de los dos rdds
rdd_3_horario_llegada_delta = rdd_3_horario_llegada_teor.join(rdd_3_horario_llegada_real)
# ('Vuelo i', (datetime_teor, datetime_real) )

rdd_3_horario_llegada_delta = rdd_3_horario_llegada_delta.map(lambda x: (
                                                            x[0], 
                                                            (
                                                                x[1][0].strftime('%Y-%m-%d %H:%M'), 
                                                                x[1][1] - x[1][0]
                                                            ) 
                                                        ))

# ('Vuelo i, (Fecha teor, timedelta)')
vuelo_mas_restrasado_llegada = rdd_3_horario_llegada_delta.reduce(lambda x, y: x if x[1][1] > y[1][1] else y)
vuelo_mas_restrasado_llegada = (vuelo_mas_restrasado_llegada[0], vuelo_mas_restrasado_llegada[1][0])
vuelo_mas_restrasado_llegada # (Vuelo i , fecha llegada teor)

('Vuelo 0', '2019-01-13 04:00')

### F. Cuál es la aerolínea más puntual. 

 Sera la aerolinea que tenga sume un timedelta menor.  
 Podemos tener en cuenta tanto horario de salida como de llegada.  
 Descartamos vuelos cancelados.

In [26]:
# Ya tengo timedelta salida, llegada por vuelo => puedo joinear y reducir por Aerolinea
rdd_3_salida_delta = rdd_3_horario_salida_delta.map(lambda x: (x[0], x[1][1]))
rdd_3_llegada_delta = rdd_3_horario_llegada_delta.map(lambda x: (x[0], x[1][1]))
rdd_3_vuelos_aerolineas = rdd_3_estado.map(lambda x: (x[0], x[1]) )

rdd_3_horario_salida_delta_aerolinea = rdd_3_vuelos_aerolineas.join(rdd_3_salida_delta)\
                                            .map( lambda x: (x[1][0], x[1][1]) )\
                                            .reduceByKey(lambda x, y: x + y)
rdd_3_horario_llegada_delta_aerolinea = rdd_3_vuelos_aerolineas.join(rdd_3_llegada_delta)\
                                            .map( lambda x: (x[1][0], x[1][1]) )\
                                            .reduceByKey(lambda x, y: x + y)
rdd_3_horario_delta_aerolinea = rdd_3_horario_salida_delta_aerolinea.union(rdd_3_horario_llegada_delta_aerolinea)\
                                    .reduceByKey( lambda x, y: x + y)
rdd_3_horario_delta_aerolinea.reduce(lambda x, y: x if x[1] < y[1] else y)[0]

'Aerolinea 1'

###  Cuál es el aeropuerto que registra mayor desviación con respecto a los horarios coordinados. 

In [27]:
rdd_3_vuelo_salida_delta = rdd_3_horario_salida_delta.map(lambda x: (x[0], x[1][1]))
rdd_3_vuelo_llegada_delta = rdd_3_horario_llegada_delta.map(lambda x: (x[0], x[1][1]))
rdd_3_vuelos_aeropuertos_salida = rdd_3_programa.map(lambda x: (x[0], x[2]) )
rdd_3_vuelos_aeropuertos_llegada = rdd_3_programa.map(lambda x: (x[0], x[3]) )

# Supongo horario coordinado de un aeropuerto como horario de salida ó llegada, 
# segun sea un aeropuerto de salida ó llegada, respectivamente 

rdd_3_aeropuerto_salida_delta = rdd_3_vuelos_aeropuertos_salida.join(rdd_3_vuelo_salida_delta)\
                                            .map( lambda x: (x[1][0], x[1][1]) )\
                                            .reduceByKey(lambda x, y: x + y)
rdd_3_aeropuerto_llegada_delta = rdd_3_vuelos_aeropuertos_llegada.join(rdd_3_vuelo_llegada_delta)\
                                            .map( lambda x: (x[1][0], x[1][1]) )\
                                            .reduceByKey(lambda x, y: x + y)
rdd_3_aeropuerto_delta = rdd_3_aeropuerto_salida_delta.union(rdd_3_aeropuerto_llegada_delta)\
                                    .reduceByKey( lambda x, y: x + y)
rdd_3_aeropuerto_delta.reduce(lambda x, y: x if x[1] > y[1] else y)[0]

'Aeropuerto llegada 4'

## Ejercicio 4 

4- Se tiene un RDD con las coordenadas de rectángulos de la forma (x1,x2,y1,y2). Se pide programar en PySpark un programa que encuentre el rectángulo de superficie mínima que contiene al punto (w,z)

In [28]:
pre_data_4_rectangulos = [
    (0,3,0,3),
    (1,2,1,2),
    (-2,4,-2,4),
    (-1,1,-1,1),
    (-1,-2,-1,-2)
]

punto_4 = (1.5,1.5)

rdd_4_rectangulos = sc.parallelize(pre_data_4_rectangulos)

In [35]:
rdd_4_rectangulos.filter(lambda x: (x[0] <= punto_4[0] <= x[1]) and (x[2] <= punto_4[1] <= x[3]))\
                    .map(lambda x: (x, (x[1]-x[0])*(x[3]-x[2]) ))\
                    .reduce(lambda x, y: x if x[1] < y[1] else y)[0]

(1, 2, 1, 2)

## Ejercicio 5 

5- Se tiene un RDD con libros en donde cada registro es un texto. Se pide obtener todos los anagramas de mas de 7 letras que puedan encontrarse. El formato de salida debe ser una lista de listas en donde cada lista tiene un conjunto de palabras que son anagramas. Ejemplo: [ [discounter, introduces, reductions], [percussion, supersonic]...

### Set de datos 

In [41]:
pre_data_5_textos = [
    ('The amazing discounter reductions'),
    ('Introduces the new anagrama'),
    ('The percussion of supersonic'),
    ('Alexander the great')
]

rdd_5_textos = sc.parallelize(pre_data_5_textos)

### Funcion auxiliar 

In [57]:
# Supongo que los textos no tienen comas ni signos de puntacion.
# Si los tiene, los tomo como parte de la palabra
def obtener_anagrama_en_orden_alfabetico(palabra):
    """
    PRE: Recibe una palabra (string)
    POST: Devuelve un anagrama de dicha palabra, 
    cuyas letras estan ordenadas alfabeticamente.
    """
    palabra_minuscula = palabra.lower()
    letras = list(palabra_minuscula)
    letras.sort()
    return ''.join(letras)
    

In [59]:
rdd_5_textos.flatMap(lambda x: x.split(' '))\
    .filter(lambda x: len(x) > 7)\
    .map( lambda x: (obtener_anagrama_en_orden_alfabetico(x), [x]) )\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda x: x[1] ).collect()

[['anagrama'],
 ['Alexander'],
 ['discounter', 'reductions', 'Introduces'],
 ['percussion', 'supersonic']]

## Ejercicio 6 

6- UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando PySpark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016.